# Dự đoán tình trạng xe với Logistic Regression

In [1]:
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt

## 1. Load và convert dữ liệu 

In [2]:
# Header: #price,maintenance,doors,persons,lug_capacity,safety,output (the condition of the car)
raw_data = np.genfromtxt('data/used_car_evaluation.csv', dtype=str, 
                         delimiter=',', skip_header=1)
print(raw_data[0])
print(raw_data[-1])
print(raw_data.shape)

['vhigh' 'vhigh' '2' '2' 'small' 'low' 'unacc']
['low' 'low' '5more' 'more' 'big' 'high' 'vgood']
(1728, 7)


In [3]:
# Kiểm tra các giá trị duy nhất của từng cột dữ liệu
# Features: 6 cột đầu
# Giá trị cần đoán: cột thứ 7 
#    (tình trạng xe: acceptable, good, unacceptable, very-good)
print(np.unique(raw_data[:,0]))
print(np.unique(raw_data[:,1]))
print(np.unique(raw_data[:,2]))
print(np.unique(raw_data[:,3]))
print(np.unique(raw_data[:,4]))
print(np.unique(raw_data[:,5]))
print(np.unique(raw_data[:,6]))

['high' 'low' 'med' 'vhigh']
['high' 'low' 'med' 'vhigh']
['2' '3' '4' '5more']
['2' '4' 'more']
['big' 'med' 'small']
['high' 'low' 'med']
['acc' 'good' 'unacc' 'vgood']


**Vì dữ liệu là dạng categorical, cần chuyển sang dạng số**

In [6]:
# Sử dụng Lớp LabelEncoder của thư viện sklearn để chuyển từng cột sang dạng số
# Lưu ý: phải thực hiện trên từng cột, vì mỗi cột là 1 feature

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()
# print(raw_data)
X = np.stack([LE.fit_transform(col) for col in raw_data.T[0:6]], axis=1)
print(X.shape)

# OHE = OneHotEncoder()
# Y = OHE.fit_transform(raw_data[:,6].reshape(-1,1)).toarray()
Y = LE.fit_transform(raw_data[:,6])
print(Y.shape)

(1728, 6)
(1728,)


In [8]:
print(X[0])
print(Y[0])

[3 3 0 0 2 1]
2


## 2. Chia dữ liệu ra thành 2 tập train và test

In [9]:
np.random.seed(3)
rand_indices = np.arange(1728)
np.random.shuffle(rand_indices)
print(rand_indices)

[ 350  112 1011 ... 1667 1273 1688]


In [10]:
N_TRAIN = 1600
train_indices = rand_indices[0:N_TRAIN]
test_indices = rand_indices[N_TRAIN:]

x_train, y_train = X[train_indices], Y[train_indices]
print(x_train.shape, y_train.shape)

x_test, y_test = X[test_indices], Y[test_indices]
print(x_test.shape, y_test.shape)

(1600, 6) (1600,)
(128, 6) (128,)


In [11]:
print(x_test[1], y_test[1])

[2 1 0 1 2 1] 2


## 3. Tạo mô hình logistic regression

In [12]:
from sklearn.linear_model import LogisticRegression

# Tạo một đối tượng regressor (mô hình dự đoán)
regressor = LogisticRegression()

# Tiến hành fit (hoặc train) mô hình để nó fit vào training data
# Thư viện sklearn sẽ tự động thực hiện việc này
regressor.fit(x_train, y_train)

# Đánh giá chất lượng mô hình với dữ liệu train
print("Training accuracy:", regressor.score(x_train, y_train))

Training accuracy: 0.7


## 4. Sử dụng mô hình

In [13]:
# Đánh giá chất lượng mô hình trên dữ liệu test
acc = regressor.score(x_test, y_test)
print("Testing accuracy: %.2f"%(acc))

Testing accuracy: 0.66


In [28]:
# Dùng mô hình để dự đoán một số sample nào đó
print(regressor.predict([x_test[18]]))

ds_loại = np.unique(raw_data[:,6])
index_loại  = regressor.predict([x_test[18]])[0]
ds_loại[index_loại]

[3]


'vgood'

In [36]:
y_test_hat = regressor.predict(x_test[0:20])
print("Dự đoán: ", y_test_hat)
print("Thực tế: ", y_test[0:20])

Dự đoán:  [2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2 3 2]
Thực tế:  [0 2 2 1 2 2 2 2 0 2 2 0 2 0 2 2 2 2 0 2]


## 5. Đánh giá mô hình với Khoảng tin cậy

In [45]:
# Confidence Interval of 95%
# Đúng về mặt thống kê, nhưng hơi khó hiểu về mặt thực tế
z = 1.96
n = y_test.shape[0]
interval = z*np.sqrt(acc*(1-acc)/n)
print(interval)
print("CI of 95%%: %.2f +/- %.2f"%(acc,interval))
print(f"Report: 95% tin cậy mô hình của chúng tôi chính xác là {acc*100:.0f}% +/- {interval*100:.0f}%")

0.08228242794922103
CI of 95%: 0.66 +/- 0.08
Report: 95% tin cậy mô hình của chúng tôi chính xác là 66% +/- 8%
